In [ ]:

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from complexPyTorch.complexLayers import ComplexLinear
import torch
from torch.utils.data import DataLoader, TensorDataset
import pennylane as qml

In [ ]:
dev = qml.device('default.qubit', wires=6)
@qml.qnode(dev)
def circuit(weights,input):
    weights = weights.view(-1, 6, 3)
    qml.AmplitudeEmbedding(features=input,normalize=True, wires=[0,1,2,3,4,5], pad_with=0)
    qml.Barrier(wires=range(6), only_visual=True)
    qml.StronglyEntanglingLayers(weights,wires=range(6))
    return  qml.state()
class  ComplexNet(nn.Module):

    def __init__(self):
        super(ComplexNet,self).__init__()
        self.weights = nn.Parameter(torch.randn(1, 6*3), requires_grad=True)
        self.com_linear1 =ComplexLinear(64, 128)
        self.com_linear2 =ComplexLinear(128, 64)
        self.com_linear3 =ComplexLinear(64, 32)
        self.com_linear4 =ComplexLinear(32,3)
    @staticmethod
    def ovonic_complex_relu(input):
        zero = torch.tensor(0)
        return torch.maximum(input.real, zero).type(torch.complex64) \
               + 1j * torch.maximum(input.imag, zero).type(torch.complex64)
    def forward(self, x):
        x = circuit(self.weights, x).type(torch.complex64)
        x = self.ovonic_complex_relu(self.com_linear1(x))
        x = self.ovonic_complex_relu(self.com_linear2(x))
        x = self.ovonic_complex_relu(self.com_linear3(x))
        x = self.com_linear4(x)
        x = (F.log_softmax(x.real, dim=1) +  F.log_softmax(x.imag, dim=1))/2
        return x

In [ ]:
k=400
epochs = 500
batch_size = 256
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 设置随机种子


with open("QCVNN_ceshi_training_metrics.txt", "w") as file:
    file.write("Fold\tEpoch\tTrain Accuracy\tTrain Loss\n")

for j in range(k):

    train_dataset = pd.read_excel(r'AM_IM除去AlCoCrFeNi1.xlsx')
    train_elements_array = np.array(train_dataset.iloc[:,4:47])
    train_encoded_labels = np.array(train_dataset['labels_encoder'])

    train_set = torch.tensor(train_elements_array, dtype=torch.float32)
    train_label = torch.tensor(train_encoded_labels, dtype=torch.long)
    train_label_dataset = TensorDataset(train_set, train_label)
    train_loader = DataLoader(train_label_dataset, batch_size=batch_size, shuffle=True)

    test_dataset_1 = pd.read_csv(r'250个AlCoCrFeNi数据集.CSV')
    test_elements_array_1 = np.array(test_dataset_1.iloc[:, 2:])

    pred_list_1 = []
    pred_list_2 = []
    model = ComplexNet().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.95)

    train_loss_list = []
    train_accuracy_list= []

    best_train_acc = 0
    best_train_loss = 0

    for epoch in range(epochs):
        train_loss = 0
        train_accuracy= 0

        avg_train_accuracy = 0
        avg_train_loss = 0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            # 计算预测结果
            preds = torch.argmax(output, dim=1)
            # 计算准确率
            train_accuracy += (preds == target).sum()
            train_loss +=loss.item()
            loss.backward()
            optimizer.step()
        avg_train_accuracy = train_accuracy / len(train_loader.sampler)
        avg_train_loss = train_loss/ len(train_loader.sampler)
        train_accuracy_list.append(avg_train_accuracy)
        train_loss_list.append(avg_train_loss)


        if best_train_acc < avg_train_accuracy:
            best_train_acc = avg_train_accuracy
            best_train_loss = avg_train_loss
            if epoch >= epochs-100 or best_train_acc>0.94:
                print('save best model best_train_acc',best_train_acc)
                torch.save(model.state_dict(), 'QCVNN_model_ceshi_'+'j_'+str(j)+'.pth')
        print('Train Epoch:{:3}  accuracy: {:.4f}  Loss: {:.4f}  '.format(
                    epoch,
                    avg_train_accuracy,
                    avg_train_loss,
                ))
        with open("QCVNN_ceshi_training_metrics.txt", "a") as file:
            file.write("{:3}\t{:3}\t{:.4f}\t\t{:.4f}\n".format(
                j, epoch, avg_train_accuracy, avg_train_loss))

    print(" j [%d] Best_Accuracy: %.4f Best_Loss: %.4f" % (j, best_train_acc, best_train_loss))

In [ ]:
pred_list_1 = []
test_dataset_1 = pd.read_csv(r'E:/桌面/250个AlCoCrFeNi数据集.CSV')
test_elements_array_1 = np.array(test_dataset_1.iloc[:, 2:])
model = ComplexNet()
model.load_state_dict(torch.load('E:/桌面/QCVNN_model_Al_content.pth'))
model.eval()
# 进行预测
output_1 = model.forward(test_elements_array_1)
y_pred_1 = torch.argmax(output_1, dim=1)
pred_list_1.append(y_pred_1)